# Up-to-date fighter data

Originally copied from Feature Additions

- This is to make sure we have up-to-date data on each fighter to reference in the final application, allowing us to not have to scrape every day, etc. 



In [1]:
# Load Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.pyplot import figure
from bs4 import BeautifulSoup
import time
import requests     # to get images
import shutil       # to save files locally
import datetime
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
from random import randint
import  random
import os
os.chdir('/Users/travisroyce/Library/CloudStorage/OneDrive-Personal/Data Science/Personal_Projects/Sports/UFC_Prediction_V2')
from cmath import nan


In [2]:
event_data = pd.read_csv('data/final/events/All_Events_Fights_and_FightUrls.csv')

These are split because they were causing problems when running all at once

New Version:

In [3]:
# Load point for fights df
fights = pd.read_csv('data/final/aggregates/Double_Fights_DF_V19.csv')

In [4]:
fights.describe()

,Unnamed: 0.1,A_Kd,B_Kd,A_Sig_strike_land,A_Sig_strike_att,B_Sig_strike_land,B_Sig_strike_att,A_Sig_strike_percent,B_Sig_strike_percent,A_Total_Strikes_land,...,A_Body_Length_to_Height_Ratio,B_Body_Length_to_Height_Ratio,Body_Length_to_Height_Ratio_Dif,A_Body_Length_to_Reach_Ratio,B_Body_Length_to_Reach_Ratio,Body_Length_to_Reach_Ratio_Dif,A_Body_Length_to_Leg_Reach_Ratio,B_Body_Length_to_Leg_Reach_Ratio,Body_Length_to_Leg_Reach_Ratio_Dif,Ruleset
count,13120.000000,13120.000000,13120.000000,13120.000000,13120.000000,13120.000000,13120.000000,13120.000000,13120.000000,13120.000000,...,12838.000000,12838.000000,1.269000e+04,12838.000000,12838.000000,1.269000e+04,12838.000000,12838.000000,1.269000e+04,13120.000000
mean,7638.432470,0.217226,0.217226,36.548018,82.508232,36.548018,82.508232,0.451466,0.451466,54.623476,...,0.427191,0.427191,1.049856e-19,0.198135,0.198135,1.791755e-17,0.747140,0.747140,1.399808e-19,2.125000
std,4421.170858,0.495956,0.495956,32.272323,70.936832,32.272323,70.936832,0.170119,0.170119,44.456560,...,0.015932,0.015932,2.229274e-02,17.478608,17.478608,2.486445e+01,0.048903,0.048903,6.851470e-02,1.058651
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.348485,0.348485,-1.149351e-01,-1400.000000,-1400.000000,-1.400448e+03,0.534884,0.534884,-3.651636e-01,1.000000
25%,3810.750000,0.000000,0.000000,12.000000,27.000000,12.000000,27.000000,0.350000,0.350000,19.000000,...,0.420000,0.420000,-1.193090e-02,0.405405,0.405405,-1.600284e-02,0.724138,0.724138,-3.618166e-02,1.000000
50%,7655.500000,0.000000,0.000000,28.000000,65.000000,28.000000,65.000000,0.450000,0.450000,46.000000,...,0.428571,0.428571,0.000000e+00,0.417219,0.417219,0.000000e+00,0.750000,0.750000,0.000000e+00,2.000000
75%,11466.250000,0.000000,0.000000,52.000000,119.000000,52.000000,119.000000,0.550000,0.550000,79.000000,...,0.434783,0.434783,1.193090e-02,0.426667,0.426667,1.600284e-02,0.769231,0.769231,3.618166e-02,3.000000
max,15311.000000,5.000000,5.000000,445.000000,744.000000,445.000000,744.000000,1.000000,1.000000,447.000000,...,0.500000,0.500000,1.149351e-01,0.503937,0.503937,1.400448e+03,1.000000,1.000000,3.651636e-01,4.000000


### Create Up-to-date Fighter Data

In [5]:
up_to_date_fighter_data = fights.copy()

# drop all columns with Rolling in the name
up_to_date_fighter_data.drop(up_to_date_fighter_data.filter(regex='Rolling').columns, axis=1, inplace=True)

up_to_date_fighter_data.head()

,Unnamed: 0.1,Fighter_A,Fighter_B,details,event_title,event_url,date,Winner,fight_id,A_Kd,...,A_Body_Length_to_Height_Ratio,B_Body_Length_to_Height_Ratio,Body_Length_to_Height_Ratio_Dif,A_Body_Length_to_Reach_Ratio,B_Body_Length_to_Reach_Ratio,Body_Length_to_Reach_Ratio_Dif,A_Body_Length_to_Leg_Reach_Ratio,B_Body_Length_to_Leg_Reach_Ratio,Body_Length_to_Leg_Reach_Ratio_Dif,Ruleset
0,0,Rick Story,Martin Kampmann,Steve English ...,UFC 139: Shogun vs Henderson,http://www.ufcstats.com/event-details/0ec82142...,2011-11-19,Martin Kampmann,89820fae001dd151,0,...,0.428571,0.430556,-0.001984,0.422535,0.418919,0.003616,0.750000,0.756098,-0.006098,1
1,1,Enrique Barzola,Kyle Bochniak,Dave Hagen ...,UFC on FOX: Maia vs. Condit,http://www.ufcstats.com/event-details/cfbccfed...,2016-08-27,Kyle Bochniak,df33799f117000cb,0,...,0.462687,0.425373,0.037313,0.442857,0.407143,0.035714,0.861111,0.740260,0.120851,2
2,2,Belal Muhammad,Takashi Sato,Rear Naked Choke,UFC 242: Khabib vs. Poirier,http://www.ufcstats.com/event-details/a79bfbc0...,2019-09-07,Belal Muhammad,18aff757c54687f1,0,...,0.436620,0.414286,0.022334,0.430556,0.397260,0.033295,0.775000,0.707317,0.067683,3
3,3,Alex Caceres,Edwin Figueroa,Two Points Deducted: Low Blows by Caceres ...,UFC 143: Diaz vs Condit,http://www.ufcstats.com/event-details/df2cf66d...,2012-02-04,Edwin Figueroa,d019250cc6d93527,0,...,0.414286,0.432836,-0.018550,0.394558,0.420290,-0.025732,0.707317,0.763158,-0.055841,1
4,4,TJ Grant,Carlo Prater,Brian Costello ...,UFC on FUEL TV: Korean Zombie vs Poirier,http://www.ufcstats.com/event-details/8caca585...,2012-05-15,TJ Grant,4655930eb83446c7,0,...,0.426197,0.426197,0.000000,0.414521,0.414521,0.000000,0.742759,0.742759,0.000000,1


In [6]:
fighter_sums = up_to_date_fighter_data.groupby('Fighter_A').sum()
fighter_sums

,Unnamed: 0.1,A_Kd,B_Kd,A_Sig_strike_land,A_Sig_strike_att,B_Sig_strike_land,B_Sig_strike_att,A_Sig_strike_percent,B_Sig_strike_percent,A_Total_Strikes_land,...,A_Body_Length_to_Height_Ratio,B_Body_Length_to_Height_Ratio,Body_Length_to_Height_Ratio_Dif,A_Body_Length_to_Reach_Ratio,B_Body_Length_to_Reach_Ratio,Body_Length_to_Reach_Ratio_Dif,A_Body_Length_to_Leg_Reach_Ratio,B_Body_Length_to_Leg_Reach_Ratio,Body_Length_to_Leg_Reach_Ratio_Dif,Ruleset
Fighter_A,,,,,,,,,,,,,,,,,,,,,
AJ Dobson,22024,0,0,177,385,251,473,1.43,1.62,230,...,1.212329,1.319831,-0.107502,1.164474,1.292044,-0.127570,2.034483,2.357601,-0.323119,11
AJ Fletcher,30696,0,0,153,313,220,405,1.91,2.19,242,...,1.885714,1.695695,0.190020,1.970149,1.617786,0.352363,3.567568,2.945809,0.621758,15
Aalon Cruz,548,0,0,2,12,20,29,0.16,0.68,2,...,0.416667,0.426471,-0.009804,0.384615,0.408451,-0.023835,0.714286,0.743590,-0.029304,3
Aaron Brink,8696,0,0,0,5,2,4,0.00,0.50,0,...,0.420000,0.413333,0.006667,0.409091,0.402597,0.006494,0.724138,0.704545,0.019592,1
Aaron Phillips,23045,0,0,62,109,129,213,1.55,1.81,346,...,1.260870,1.266807,-0.005937,1.225352,1.255773,-0.030421,2.175000,2.193590,-0.018590,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zhalgas Zhumagulov,57025,0,2,388,875,419,845,2.64,2.88,433,...,2.625000,2.552675,0.072325,2.526316,2.489668,0.036647,4.666667,4.456127,0.210540,20
Zhang Lipeng,9016,0,0,48,93,67,116,1.02,1.11,98,...,0.852394,0.822938,0.029457,0.852394,0.799432,0.052962,1.485518,1.401163,0.084355,4
Zhang Weili,62276,2,1,504,911,198,482,4.68,3.40,800,...,3.500000,3.375310,0.124690,3.555556,3.355096,0.200460,6.222222,5.851128,0.371094,26


#### Add Up-to-date rolling for each metric 

In [7]:
# groupby Fighter_A and get each column mean and std
fighter_averages = up_to_date_fighter_data.groupby('Fighter_A').mean().reset_index()
fighter_averages

,Fighter_A,Unnamed: 0.1,A_Kd,B_Kd,A_Sig_strike_land,A_Sig_strike_att,B_Sig_strike_land,B_Sig_strike_att,A_Sig_strike_percent,B_Sig_strike_percent,...,A_Body_Length_to_Height_Ratio,B_Body_Length_to_Height_Ratio,Body_Length_to_Height_Ratio_Dif,A_Body_Length_to_Reach_Ratio,B_Body_Length_to_Reach_Ratio,Body_Length_to_Reach_Ratio_Dif,A_Body_Length_to_Leg_Reach_Ratio,B_Body_Length_to_Leg_Reach_Ratio,Body_Length_to_Leg_Reach_Ratio_Dif,Ruleset
0,AJ Dobson,7341.333333,0.000,0.000000,59.000000,128.333333,83.666667,157.666667,0.476667,0.540000,...,0.404110,0.439944,-0.035834,0.388158,0.430681,-0.042523,0.678161,0.785867,-0.107706,3.666667
1,AJ Fletcher,7674.000000,0.000,0.000000,38.250000,78.250000,55.000000,101.250000,0.477500,0.547500,...,0.471429,0.423924,0.047505,0.492537,0.404447,0.088091,0.891892,0.736452,0.155440,3.750000
2,Aalon Cruz,548.000000,0.000,0.000000,2.000000,12.000000,20.000000,29.000000,0.160000,0.680000,...,0.416667,0.426471,-0.009804,0.384615,0.408451,-0.023835,0.714286,0.743590,-0.029304,3.000000
3,Aaron Brink,8696.000000,0.000,0.000000,0.000000,5.000000,2.000000,4.000000,0.000000,0.500000,...,0.420000,0.413333,0.006667,0.409091,0.402597,0.006494,0.724138,0.704545,0.019592,1.000000
4,Aaron Phillips,7681.666667,0.000,0.000000,20.666667,36.333333,43.000000,71.000000,0.516667,0.603333,...,0.420290,0.422269,-0.001979,0.408451,0.418591,-0.010140,0.725000,0.731197,-0.006197,1.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2138,Zhalgas Zhumagulov,9504.166667,0.000,0.333333,64.666667,145.833333,69.833333,140.833333,0.440000,0.480000,...,0.437500,0.425446,0.012054,0.421053,0.414945,0.006108,0.777778,0.742688,0.035090,3.333333
2139,Zhang Lipeng,4508.000000,0.000,0.000000,24.000000,46.500000,33.500000,58.000000,0.510000,0.555000,...,0.426197,0.411469,0.014728,0.426197,0.399716,0.026481,0.742759,0.700581,0.042178,2.000000
2140,Zhang Weili,7784.500000,0.250,0.125000,63.000000,113.875000,24.750000,60.250000,0.585000,0.425000,...,0.437500,0.421914,0.015586,0.444444,0.419387,0.025057,0.777778,0.731391,0.046387,3.250000
2141,Zubaira Tukhugov,7941.125000,0.375,0.000000,40.250000,99.875000,39.875000,107.375000,0.428750,0.361250,...,0.433824,0.423855,0.009969,0.433824,0.404435,0.029388,0.766234,0.736551,0.029682,2.625000


In [8]:
fighter_averages.columns.to_list()

['Fighter_A',
 'Unnamed: 0.1',
 'A_Kd',
 'B_Kd',
 'A_Sig_strike_land',
 'A_Sig_strike_att',
 'B_Sig_strike_land',
 'B_Sig_strike_att',
 'A_Sig_strike_percent',
 'B_Sig_strike_percent',
 'A_Total_Strikes_land',
 'A_Total_Strikes_att',
 'B_Total_Strikes_land',
 'B_Total_Strikes_att',
 'A_Total_Strikes_percent',
 'B_Total_Strikes_percent',
 'A_Takedowns_land',
 'A_Takedowns_att',
 'B_Takedowns_land',
 'B_Takedowns_att',
 'A_Takedown_percent',
 'B_Takedown_percent',
 'A_Sub_Attempts_land',
 'A_Sub_Attempts_att',
 'B_Sub_Attempts_land',
 'B_Sub_Attempts_att',
 'A_Sub_Success_Percent',
 'B_Sub_Success_Percent',
 'A_Rev',
 'B_Rev',
 'A_Ctrl_time_min',
 'A_Ctrl_time_sec',
 'B_Ctrl_time_min',
 'B_Ctrl_time_sec',
 'A_Ctrl_time_tot',
 'B_Ctrl_time_tot',
 'A_Head_Strikes_land',
 'A_Head_Strikes_att',
 'B_Head_Strikes_land',
 'B_Head_Strikes_att',
 'A_Head_Strikes_percent',
 'B_Head_Strikes_percent',
 'A_Body_Strikes_land',
 'A_Body_Strikes_att',
 'B_Body_Strikes_land',
 'B_Body_Strikes_att',
 'A_B

In [9]:
fighterA_std = up_to_date_fighter_data.groupby('Fighter_A').std().reset_index()

## Add Tapology Data

This includes:
- UFC Win-by %s (KO, SUB, DEC)
- UFC Lose-by %s ('')
- Previous Organizations
- Previous Record
- Previous Win-by %s
- Previous Lose-by %s


Data Stored: data/tapology/fighters

In [10]:
fights.head()

,Unnamed: 0.1,Fighter_A,Fighter_B,details,event_title,event_url,date,Winner,fight_id,A_Kd,...,A_Body_Length_to_Height_Ratio,B_Body_Length_to_Height_Ratio,Body_Length_to_Height_Ratio_Dif,A_Body_Length_to_Reach_Ratio,B_Body_Length_to_Reach_Ratio,Body_Length_to_Reach_Ratio_Dif,A_Body_Length_to_Leg_Reach_Ratio,B_Body_Length_to_Leg_Reach_Ratio,Body_Length_to_Leg_Reach_Ratio_Dif,Ruleset
0,0,Rick Story,Martin Kampmann,Steve English ...,UFC 139: Shogun vs Henderson,http://www.ufcstats.com/event-details/0ec82142...,2011-11-19,Martin Kampmann,89820fae001dd151,0,...,0.428571,0.430556,-0.001984,0.422535,0.418919,0.003616,0.750000,0.756098,-0.006098,1
1,1,Enrique Barzola,Kyle Bochniak,Dave Hagen ...,UFC on FOX: Maia vs. Condit,http://www.ufcstats.com/event-details/cfbccfed...,2016-08-27,Kyle Bochniak,df33799f117000cb,0,...,0.462687,0.425373,0.037313,0.442857,0.407143,0.035714,0.861111,0.740260,0.120851,2
2,2,Belal Muhammad,Takashi Sato,Rear Naked Choke,UFC 242: Khabib vs. Poirier,http://www.ufcstats.com/event-details/a79bfbc0...,2019-09-07,Belal Muhammad,18aff757c54687f1,0,...,0.436620,0.414286,0.022334,0.430556,0.397260,0.033295,0.775000,0.707317,0.067683,3
3,3,Alex Caceres,Edwin Figueroa,Two Points Deducted: Low Blows by Caceres ...,UFC 143: Diaz vs Condit,http://www.ufcstats.com/event-details/df2cf66d...,2012-02-04,Edwin Figueroa,d019250cc6d93527,0,...,0.414286,0.432836,-0.018550,0.394558,0.420290,-0.025732,0.707317,0.763158,-0.055841,1
4,4,TJ Grant,Carlo Prater,Brian Costello ...,UFC on FUEL TV: Korean Zombie vs Poirier,http://www.ufcstats.com/event-details/8caca585...,2012-05-15,TJ Grant,4655930eb83446c7,0,...,0.426197,0.426197,0.000000,0.414521,0.414521,0.000000,0.742759,0.742759,0.000000,1


Get Fighter Tapology

Now, we need to, for each fighter, get: 

- UFC Fight # [All_Fight_Results]
- UFC Win %   [All_Fight_Results]
- Previous Fight Organization [details]
- UFC Win % (KO, Sub, Dec) [All_Fight_Results]
- UFC Lose % (KO, Sub, Dec) [All_Fight_Results]
- Previous Org Win % (Ko, Sub, Dec) [All_Fight_Results]
- Previous Org Lose % (Ko, Sub, Dec) [All_Fight_Results]
- Fight Camp [details]
- Fighting Out Of [details]

FROM the fight_results_df. 

In [11]:
fight_results_df = pd.read_csv('data/tapology/aggregates/all_fight_results.csv')

In [12]:
# correct dates of fights by using datetime
fight_results_df['Fight Date'] = pd.to_datetime(fight_results_df['Fight Date'])
fight_results_df['Fight Date'] = fight_results_df['Fight Date'].dt.date
# sort by date
fight_results_df = fight_results_df.sort_values(by=['Fight Date'], ascending=False)
today = datetime.date.today()
# filter out fights that haven't happened yet
fight_results_df = fight_results_df[fight_results_df['Fight Date'] < today]
# drop Confirmed Upcoming Bouts
fight_results_df = fight_results_df[fight_results_df['Fight Summary'] != 'Confirmed Upcoming Bout']
# drop Cancelled Bouts
fight_results_df = fight_results_df[fight_results_df['Fight Summary'] != 'Cancelled Bout']

fight_results_df.head()

,Opponent Name,Fighter Record Before Fight,Opponent Record Before Fight,Fight Summary,Event,Fight Date,Fighter Name
104226,Eduardo Matias Torres Caut,23-4-0,16-1-0,Win · Kneebar · 2:16 · R2,Contender Series 2023,2023-10-03,Victor Hugo
19578,Gregg Ellis,12-4-0,8-4-0,Win · Decision · Unanimous,CFFC 124: Ellis vs. Daukaus,2023-09-02,Kyle Daukaus
58916,Alison Vicente,14-9-0,27-22-0,Win · Decision · Unanimous,Federação Fight 16,2023-09-02,Luis Henrique
8153,Benoit Saint-Denis,17-6-0,11-1-0,Loss · Ground & Pound · 4:44 · R2,UFC Fight Night,2023-09-02,Thiago Moisés
38063,Benoit Saint-Denis,17-6-0,11-1-0,Loss · Ground & Pound · 4:44 · R2,UFC Fight Night,2023-09-02,Thiago Moises


In [13]:
today = datetime.date.today()

In [14]:
def get_tapology_data(fighter_name, date, chosen_stat):
        # Chosen Stat can be ufc_fights, ufc_Wins, ufc_losses, ufc_decision_wins, ufc_decision_losses, ufc_ko_wins, ufc_sub_wins, ufc_sub_losses, ufc_tko_losses, 
        # ufc_win_percent, ufc_ko_win_percent, ufc_sub_win_percent, ufc_decision_win_percent, ufc_loss_percent, ufc_ko_loss_percent, ufc_sub_loss_percent, 
        # ufc_decision_loss_percent, previous_promotion, previous_promotion_fights, previous_promotion_win_percent, previous_promotion_ko_win_percent, 
        # previous_promotion_sub_win_percent, previous_promotion_decision_win_percent, previous_promotion_loss_percent, previous_promotion_ko_loss_percent, 
        # previous_promotion_sub_loss_percent, previous_promotion_decision_loss_percent, affiliation, fighting_out_of
    try:

        # File paths
        try:
            results_file = f'data/tapology/fighters/{fighter_name}_fight_results.csv'
            record_by_promotion_file = f'data/tapology/fighters/{fighter_name}_record_by_promotion.csv'
            details_file = f'data/tapology/fighters/{fighter_name}_details.csv'

            # Read
            fighter_results = pd.read_csv(results_file)
            fighter_results['Fighter Name'] = fighter_name

        except FileNotFoundError as e:
            return f"Error: {e}. Check if the file exists and the path is correct."

        except ValueError as e:
            return f"Error: Invalid value encountered. Details: {e}"

        except Exception as e:
            return f"An unexpected error occurred: {e}"

        # Make sure fighter results contain 'Fight Date' column. If not, return error
        if 'Fight Date' not in fighter_results.columns:
            return f"Error: {results_file} does not contain 'Fight Date' column. Check if the file is correct."
            

        # Filtering UFC fights before date
        chosen_date = datetime.datetime.strptime(date, '%Y-%m-%d').date()
        fighter_results['Fight Date'] = pd.to_datetime(fighter_results['Fight Date']).dt.date
        ufc_fights_df = fighter_results[(fighter_results['Fight Date'] < chosen_date) & 
                                        fighter_results['Event'].astype(str).str.contains('UFC') & 
                                        (fighter_results['Fight Summary'] != 'Cancelled Bout')]
        
        # Calculating UFC stats
        # ufc fights
        ufc_fights = ufc_fights_df.shape[0]
        # ufc wins
        ufc_wins = ufc_fights_df[ufc_fights_df['Fight Summary'].str.contains('Win')].shape[0]
        ufc_losses = ufc_fights_df[ufc_fights_df['Fight Summary'].str.contains('Loss')].shape[0]
        ufc_decision_wins = ufc_fights_df[ufc_fights_df['Fight Summary'].str.contains('Win · Decision')].shape[0]
        ufc_decision_losses = ufc_fights_df[ufc_fights_df['Fight Summary'].str.contains('Loss · Decision')].shape[0]
        ufc_fights_df['Win/Loss'] = ufc_fights_df['Fight Summary'].str.split(' ').str[0]
        ufc_fights_df['Fight Sum'] = ufc_fights_df['Fight Summary'].str.split('·').str[:2].str.join(', ')
        ufc_fights_df['Ending_Method'] = ufc_fights_df['Fight Sum'].str.split(', ').str[1] 

        # add sub list for calculating 
        submission_list = ['Guillotine', 'Guillotine Choke', 'Choke', 'Rear Naked Choke', 'Americana', 'Kimura', 'Heel Hook', 'Triangle', 'Bulldog', 'Von Flue', 'Armbar']
        ufc_fights_df['Is_Submission'] = ufc_fights_df['Ending_Method'].apply(lambda x: any(sub in str(x) for sub in submission_list) if pd.notnull(x) else False)
        ufc_fights_df['Finish'] = np.where(ufc_fights_df['Is_Submission'], 'Submission', np.where(ufc_fights_df['Ending_Method'].str.contains('Decision'), 'Decision', 'KO/TKO'))
        ufc_fights_df['Finish'] = np.where(ufc_fights_df['Ending_Method'].str.contains('Decision'), 'Decision', ufc_fights_df['Finish'])

        # UFC KO Wins
        ufc_ko_wins = ufc_fights_df[ufc_fights_df['Finish'] == 'KO/TKO'].shape[0]
        ufc_sub_wins = ufc_fights_df[(ufc_fights_df['Win/Loss'] == 'Win') & (ufc_fights_df['Finish'] == 'Submission')].shape[0]
        ufc_sub_losses = ufc_fights_df[(ufc_fights_df['Win/Loss'] == 'Loss') & (ufc_fights_df['Finish'] == 'Submission')].shape[0]
        ufc_tko_losses = ufc_fights_df[(ufc_fights_df['Win/Loss'] == 'Loss') & (ufc_fights_df['Finish'] == 'KO/TKO')].shape[0]
        ufc_win_percent = ufc_wins / ufc_fights if ufc_fights != 0 else 0
        ufc_ko_win_percent = ufc_ko_wins / ufc_fights if ufc_fights != 0 else 0
        ufc_sub_win_percent = ufc_sub_wins / ufc_fights if ufc_fights != 0 else 0
        ufc_decision_win_percent = ufc_decision_wins / ufc_fights if ufc_fights != 0 else 0
        ufc_loss_percent = ufc_losses / ufc_fights if ufc_fights != 0 else 0
        ufc_ko_loss_percent = ufc_tko_losses / ufc_fights if ufc_fights != 0 else 0
        ufc_sub_loss_percent = ufc_sub_losses / ufc_fights if ufc_fights != 0 else 0
        ufc_decision_loss_percent = ufc_decision_losses / ufc_fights if ufc_fights != 0 else 0

        ufc_stats = pd.DataFrame({
            'Fighter': fighter_name,
            'UFC Fights': ufc_fights,
            'UFC Wins': ufc_wins,
            'UFC Losses': ufc_losses,
            'UFC Win %': ufc_win_percent,
            'UFC KO/TKO Wins': ufc_ko_wins,
            'UFC Submission Wins': ufc_sub_wins,
            'UFC Decision Wins': ufc_decision_wins,
            'UFC KO/TKO Losses': ufc_tko_losses,
            'UFC Submission Losses': ufc_sub_losses,
            'UFC Decision Losses': ufc_decision_losses,
            'UFC KO/TKO Win %': ufc_ko_win_percent,
            'UFC Submission Win %': ufc_sub_win_percent,
            'UFC Decision Win %': ufc_decision_win_percent,
            'UFC KO/TKO Loss %': ufc_ko_loss_percent,
            'UFC Submission Loss %': ufc_sub_loss_percent,
            'UFC Decision Loss %': ufc_decision_loss_percent
        }, index=[0])

        
        # Load Record by Promotion
        record_by_promotion = pd.read_csv(record_by_promotion_file)
        non_ufc_promotions = record_by_promotion[~record_by_promotion['Promotion'].str.contains('UFC')]
        non_ufc_promotions = non_ufc_promotions.sort_values(by=['Wins'], ascending=False)
        non_ufc_promotions = non_ufc_promotions.head(1)
        try:
            previous_promotion = non_ufc_promotions['Promotion'].tolist()[0]
            denominator = non_ufc_promotions['Wins'].tolist()[0] + non_ufc_promotions['Losses'].tolist()[0]
            previous_promotion_win_percent = non_ufc_promotions['Wins'].tolist()[0] / denominator if denominator != 0 else 0
            previous_promotion_fights = non_ufc_promotions['Wins'].tolist()[0] + non_ufc_promotions['Losses'].tolist()[0]
            decision_win_percent = non_ufc_promotions['Decision Win Percent'].tolist()[0]
            ko_win_percent = non_ufc_promotions['KO/TKO Win Percent'].tolist()[0]
            sub_win_percent = non_ufc_promotions['Sub Win Percent'].tolist()[0]
            decision_loss_percent = non_ufc_promotions['Decision Loss Percent'].tolist()[0]
            ko_loss_percent = non_ufc_promotions['KO/TKO Loss Percent'].tolist()[0]
            sub_loss_percent = non_ufc_promotions['Sub Loss Percent'].tolist()[0]
            previous_promotion_stats = pd.DataFrame({
                'Fighter': fighter_name,
                'Previous Promotion': previous_promotion,
                'Previous Promotion Fights': previous_promotion_fights,
                'Previous Promotion Win %': previous_promotion_win_percent,
                'Previous Promotion KO/TKO Win %': ko_win_percent,
                'Previous Promotion Submission Win %': sub_win_percent,
                'Previous Promotion Decision Win %': decision_win_percent,
                'Previous Promotion KO/TKO Loss %': ko_loss_percent,
                'Previous Promotion Submission Loss %': sub_loss_percent,
                'Previous Promotion Decision Loss %': decision_loss_percent
            }, index=[0])
        
        except IndexError:
            previous_promotion = None
            previous_promotion_stats = pd.DataFrame({
                'Fighter': fighter_name,
                'Previous Promotion': 'None',
                'Previous Promotion Fights': np.nan,
                'Previous Promotion Win %': np.nan,
                'Previous Promotion KO/TKO Win %': np.nan,
                'Previous Promotion Submission Win %': np.nan,
                'Previous Promotion Decision Win %': np.nan,
                'Previous Promotion KO/TKO Loss %': np.nan,
                'Previous Promotion Submission Loss %': np.nan,
                'Previous Promotion Decision Loss %': np.nan
            }, index=[0])

            


        # Load Details
        details = pd.read_csv(details_file)
        try:
            affiliation = details['Affiliation'].tolist()[0]
        except:
            affiliation = None
        try:
            fighting_out_of = details['Fighting out of'].tolist()[0]
        except:
            fighting_out_of = None

        affiliation_stats = pd.DataFrame({
            'Fighter': fighter_name,
            'Affiliation': affiliation,
            'Fighting Out Of': fighting_out_of
        }, index=[0])

        # Merge all stats
        all_stats = pd.merge(ufc_stats, previous_promotion_stats, on='Fighter', how='left')
        all_stats = pd.merge(all_stats, affiliation_stats, on='Fighter', how='left')

        # All Stats
        fights = all_stats['UFC Fights'].tolist()[0]
        wins = all_stats['UFC Wins'].tolist()[0]
        losses = all_stats['UFC Losses'].tolist()[0]
        win_percent = all_stats['UFC Win %'].tolist()[0]
        ko_wins = all_stats['UFC KO/TKO Wins'].tolist()[0]
        sub_wins = all_stats['UFC Submission Wins'].tolist()[0]
        decision_wins = all_stats['UFC Decision Wins'].tolist()[0]
        ko_losses = all_stats['UFC KO/TKO Losses'].tolist()[0]
        sub_losses = all_stats['UFC Submission Losses'].tolist()[0]
        decision_losses = all_stats['UFC Decision Losses'].tolist()[0]
        ko_win_percent = all_stats['UFC KO/TKO Win %'].tolist()[0]
        sub_win_percent = all_stats['UFC Submission Win %'].tolist()[0]
        decision_win_percent = all_stats['UFC Decision Win %'].tolist()[0]
        ko_loss_percent = all_stats['UFC KO/TKO Loss %'].tolist()[0]
        sub_loss_percent = all_stats['UFC Submission Loss %'].tolist()[0]
        decision_loss_percent = all_stats['UFC Decision Loss %'].tolist()[0]
        previous_promotion = all_stats['Previous Promotion'].tolist()[0]
        previous_promotion_fights = all_stats['Previous Promotion Fights'].tolist()[0]
        previous_promotion_win_percent = all_stats['Previous Promotion Win %'].tolist()[0]
        previous_promotion_ko_win_percent = all_stats['Previous Promotion KO/TKO Win %'].tolist()[0]
        previous_promotion_sub_win_percent = all_stats['Previous Promotion Submission Win %'].tolist()[0]
        previous_promotion_decision_win_percent = all_stats['Previous Promotion Decision Win %'].tolist()[0]
        previous_promotion_ko_loss_percent = all_stats['Previous Promotion KO/TKO Loss %'].tolist()[0]
        previous_promotion_sub_loss_percent = all_stats['Previous Promotion Submission Loss %'].tolist()[0]
        previous_promotion_decision_loss_percent = all_stats['Previous Promotion Decision Loss %'].tolist()[0]
        affiliation = all_stats['Affiliation'].tolist()[0]
        fighting_out_of = all_stats['Fighting Out Of'].tolist()[0]

        if chosen_stat == 'ufc_fights':
            return fights
        elif chosen_stat == 'ufc_wins':
            return wins
        elif chosen_stat == 'ufc_losses':
            return losses
        elif chosen_stat == 'ufc_win_percent':
            return win_percent
        elif chosen_stat == 'ufc_ko_wins':
            return ko_wins
        elif chosen_stat == 'ufc_sub_wins':
            return sub_wins
        elif chosen_stat == 'ufc_decision_wins':
            return decision_wins
        elif chosen_stat == 'ufc_ko_losses':
            return ko_losses
        elif chosen_stat == 'ufc_sub_losses':
            return sub_losses
        elif chosen_stat == 'ufc_decision_losses':
            return decision_losses
        elif chosen_stat == 'ufc_ko_win_percent':
            return ko_win_percent
        elif chosen_stat == 'ufc_sub_win_percent':
            return sub_win_percent
        elif chosen_stat == 'ufc_decision_win_percent':
            return decision_win_percent
        elif chosen_stat == 'ufc_ko_loss_percent':
            return ko_loss_percent
        elif chosen_stat == 'ufc_sub_loss_percent':
            return sub_loss_percent
        elif chosen_stat == 'ufc_decision_loss_percent':
            return decision_loss_percent
        elif chosen_stat == 'previous_promotion':
            return previous_promotion
        elif chosen_stat == 'previous_promotion_fights':
            return previous_promotion_fights
        elif chosen_stat == 'previous_promotion_win_percent':
            return previous_promotion_win_percent
        elif chosen_stat == 'previous_promotion_ko_win_percent':
            return previous_promotion_ko_win_percent
        elif chosen_stat == 'previous_promotion_sub_win_percent':
            return previous_promotion_sub_win_percent
        elif chosen_stat == 'previous_promotion_decision_win_percent':
            return previous_promotion_decision_win_percent
        elif chosen_stat == 'previous_promotion_ko_loss_percent':
            return previous_promotion_ko_loss_percent
        elif chosen_stat == 'previous_promotion_sub_loss_percent':
            return previous_promotion_sub_loss_percent
        elif chosen_stat == 'previous_promotion_decision_loss_percent':
            return previous_promotion_decision_loss_percent
        elif chosen_stat == 'affiliation':
            return affiliation
        elif chosen_stat == 'fighting_out_of':
            return fighting_out_of
 
    except:
        return np.nan

In [15]:
today = datetime.date.today().strftime('%Y-%m-%d')
today

'2023-10-27'

In [16]:
fighter_name = "Israel Adesanya"  # You can replace this with any fighter's name for whom you have the required CSV data.
stats = get_tapology_data(fighter_name, today, 'ufc_fights')
stats

15

In [17]:
fighter_averages['UFC_Matches_Total'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'ufc_fights'), axis=1)
# wins total
fighter_averages['UFC_Wins_Total'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'ufc_wins'), axis=1)
# losses total
fighter_averages['UFC_Losses_Total'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'ufc_losses'), axis=1)
# win percent
fighter_averages['UFC_Win_Percent'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'ufc_win_percent'), axis=1)
# ko wins
fighter_averages['UFC_KO_Wins_Total'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'ufc_ko_wins'), axis=1)
# sub wins
fighter_averages['UFC_Sub_Wins_Total'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'ufc_sub_wins'), axis=1)
# decision wins
fighter_averages['UFC_Decision_Wins_Total'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'ufc_decision_wins'), axis=1)
# ko losses
fighter_averages['UFC_KO_Losses_Total'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'ufc_ko_losses'), axis=1)
# sub losses
fighter_averages['UFC_Sub_Losses_Total'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'ufc_sub_losses'), axis=1)
# decision losses
fighter_averages['UFC_Decision_Losses_Total'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'ufc_decision_losses'), axis=1)
# ko win percent
fighter_averages['UFC_KO_Win_Percent'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'ufc_ko_win_percent'), axis=1)
# sub win percent
fighter_averages['UFC_Sub_Win_Percent'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'ufc_sub_win_percent'), axis=1)
# decision win percent
fighter_averages['UFC_Decision_Win_Percent'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'ufc_decision_win_percent'), axis=1)
# ko loss percent
fighter_averages['UFC_KO_Loss_Percent'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'ufc_ko_loss_percent'), axis=1)
# sub loss percent
fighter_averages['UFC_Sub_Loss_Percent'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'ufc_sub_loss_percent'), axis=1)
# decision loss percent
fighter_averages['UFC_Decision_Loss_Percent'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'ufc_decision_loss_percent'), axis=1)
# previous promotion
fighter_averages['Previous_Promotion'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'previous_promotion'), axis=1)
# previous promotion fights
fighter_averages['Previous_Promotion_Fights'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'previous_promotion_fights'), axis=1)
# previous promotion win percent
fighter_averages['Previous_Promotion_Win_Percent'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'previous_promotion_win_percent'), axis=1)
# previous promotion ko win percent
fighter_averages['Previous_Promotion_KO_Win_Percent'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'previous_promotion_ko_win_percent'), axis=1)
# previous promotion sub win percent
fighter_averages['Previous_Promotion_Sub_Win_Percent'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'previous_promotion_sub_win_percent'), axis=1)
# previous promotion decision win percent
fighter_averages['Previous_Promotion_Decision_Win_Percent'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'previous_promotion_decision_win_percent'), axis=1)
# previous promotion ko loss percent
fighter_averages['Previous_Promotion_KO_Loss_Percent'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'previous_promotion_ko_loss_percent'), axis=1)
# previous promotion sub loss percent
fighter_averages['Previous_Promotion_Sub_Loss_Percent'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'previous_promotion_sub_loss_percent'), axis=1)
# previous promotion decision loss percent
fighter_averages['Previous_Promotion_Decision_Loss_Percent'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'previous_promotion_decision_loss_percent'), axis=1)
# affiliation
fighter_averages['Affiliation'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'affiliation'), axis=1)
# fighting out of
fighter_averages['Fighting_Out_Of'] = fighter_averages.apply(lambda row: get_tapology_data(row['Fighter_A'], today, 'fighting_out_of'), axis=1)

fighter_averages.head()

,Fighter_A,Unnamed: 0.1,A_Kd,B_Kd,A_Sig_strike_land,A_Sig_strike_att,B_Sig_strike_land,B_Sig_strike_att,A_Sig_strike_percent,B_Sig_strike_percent,...,Previous_Promotion_Fights,Previous_Promotion_Win_Percent,Previous_Promotion_KO_Win_Percent,Previous_Promotion_Sub_Win_Percent,Previous_Promotion_Decision_Win_Percent,Previous_Promotion_KO_Loss_Percent,Previous_Promotion_Sub_Loss_Percent,Previous_Promotion_Decision_Loss_Percent,Affiliation,Fighting_Out_Of
0,AJ Dobson,7341.333333,0.0,0.0,59.000000,128.333333,83.666667,157.666667,0.476667,0.540000,...,Error: [Errno 2] No such file or directory: 'd...,Error: [Errno 2] No such file or directory: 'd...,Error: [Errno 2] No such file or directory: 'd...,Error: [Errno 2] No such file or directory: 'd...,Error: [Errno 2] No such file or directory: 'd...,Error: [Errno 2] No such file or directory: 'd...,Error: [Errno 2] No such file or directory: 'd...,Error: [Errno 2] No such file or directory: 'd...,Error: [Errno 2] No such file or directory: 'd...,Error: [Errno 2] No such file or directory: 'd...
1,AJ Fletcher,7674.000000,0.0,0.0,38.250000,78.250000,55.000000,101.250000,0.477500,0.547500,...,1,1.0,100%,0,0,0,0,0,Gladiators Academy,"Lafayette, Louisiana"
2,Aalon Cruz,548.000000,0.0,0.0,2.000000,12.000000,20.000000,29.000000,0.160000,0.680000,...,2,0.5,0,0,100%,0,0,100%,Kaizen MMA & Yoga,"Tampa, Florida"
3,Aaron Brink,8696.000000,0.0,0.0,0.000000,5.000000,2.000000,4.000000,0.000000,0.500000,...,2,0.5,100%,0,0,0,100%,0,NaN,"San Diego, California, USA"
4,Aaron Phillips,7681.666667,0.0,0.0,20.666667,36.333333,43.000000,71.000000,0.516667,0.603333,...,6,0.833333,60%,20%,20%,100%,0,0,Headkicks MMA,"Lafayette, Louisiana"


In [18]:
# check new columns to ensure they have correct dtypes
new_cols = fighter_averages.columns[-27:]
new_cols

Index(['UFC_Matches_Total', 'UFC_Wins_Total', 'UFC_Losses_Total',
       'UFC_Win_Percent', 'UFC_KO_Wins_Total', 'UFC_Sub_Wins_Total',
       'UFC_Decision_Wins_Total', 'UFC_KO_Losses_Total',
       'UFC_Sub_Losses_Total', 'UFC_Decision_Losses_Total',
       'UFC_KO_Win_Percent', 'UFC_Sub_Win_Percent', 'UFC_Decision_Win_Percent',
       'UFC_KO_Loss_Percent', 'UFC_Sub_Loss_Percent',
       'UFC_Decision_Loss_Percent', 'Previous_Promotion',
       'Previous_Promotion_Fights', 'Previous_Promotion_Win_Percent',
       'Previous_Promotion_KO_Win_Percent',
       'Previous_Promotion_Sub_Win_Percent',
       'Previous_Promotion_Decision_Win_Percent',
       'Previous_Promotion_KO_Loss_Percent',
       'Previous_Promotion_Sub_Loss_Percent',
       'Previous_Promotion_Decision_Loss_Percent', 'Affiliation',
       'Fighting_Out_Of'],
      dtype='object')

In [19]:
# get dtypes of new_cols
fighter_averages[new_cols].dtypes

UFC_Matches_Total                           object
UFC_Wins_Total                              object
UFC_Losses_Total                            object
UFC_Win_Percent                             object
UFC_KO_Wins_Total                           object
UFC_Sub_Wins_Total                          object
UFC_Decision_Wins_Total                     object
UFC_KO_Losses_Total                         object
UFC_Sub_Losses_Total                        object
UFC_Decision_Losses_Total                   object
UFC_KO_Win_Percent                          object
UFC_Sub_Win_Percent                         object
UFC_Decision_Win_Percent                    object
UFC_KO_Loss_Percent                         object
UFC_Sub_Loss_Percent                        object
UFC_Decision_Loss_Percent                   object
Previous_Promotion                          object
Previous_Promotion_Fights                   object
Previous_Promotion_Win_Percent              object
Previous_Promotion_KO_Win_Perce

In [20]:
# do the same with the rest of the new columns except for A_Previous_Promotion, B_Previous_Promotion, A_Affiliation, B_Affiliation, A_Fighting_Out_Of, B_Fighting_Out_Of
for col in new_cols:
    if col not in ['Previous_Promotion', 'Affiliation',  'Fighting_Out_Of']:
        fighter_averages[col] = pd.to_numeric(fighter_averages[col], errors='coerce')

In [21]:
# check dtypes again
fighter_averages[new_cols].dtypes

UFC_Matches_Total                           float64
UFC_Wins_Total                              float64
UFC_Losses_Total                            float64
UFC_Win_Percent                             float64
UFC_KO_Wins_Total                           float64
UFC_Sub_Wins_Total                          float64
UFC_Decision_Wins_Total                     float64
UFC_KO_Losses_Total                         float64
UFC_Sub_Losses_Total                        float64
UFC_Decision_Losses_Total                   float64
UFC_KO_Win_Percent                          float64
UFC_Sub_Win_Percent                         float64
UFC_Decision_Win_Percent                    float64
UFC_KO_Loss_Percent                         float64
UFC_Sub_Loss_Percent                        float64
UFC_Decision_Loss_Percent                   float64
Previous_Promotion                           object
Previous_Promotion_Fights                   float64
Previous_Promotion_Win_Percent              float64
Previous_Pro

In [22]:
# count of each fighter
fighter_counts = up_to_date_fighter_data.groupby('Fighter_A').count().reset_index()
# just keep first 3 columns
fighter_counts = fighter_counts.iloc[:,0:3]
unnamed_cols = [n for n in fighter_counts.columns if 'Unnamed' in n]
# drop
fighter_counts.drop(unnamed_cols, axis=1, inplace=True)

# rename Fighter_B to UFC_Fight_Count
fighter_counts = fighter_counts.rename(columns={'Fighter_B': 'UFC_Fight_Count'})
fighter_counts.head()

,Fighter_A,UFC_Fight_Count
0,AJ Dobson,3
1,AJ Fletcher,4
2,Aalon Cruz,1
3,Aaron Brink,1
4,Aaron Phillips,3


In [23]:
# find Jailton Almedia
fighter_counts[fighter_counts['Fighter_A'] == 'Jailton Almeida']

,Fighter_A,UFC_Fight_Count
842,Jailton Almeida,6


In [24]:
# add count of fights to fighter_averages by Fighter_A
fighter_averages = pd.merge(fighter_averages, fighter_counts, on='Fighter_A', how='left')
fighter_averages.head()

,Fighter_A,Unnamed: 0.1,A_Kd,B_Kd,A_Sig_strike_land,A_Sig_strike_att,B_Sig_strike_land,B_Sig_strike_att,A_Sig_strike_percent,B_Sig_strike_percent,...,Previous_Promotion_Win_Percent,Previous_Promotion_KO_Win_Percent,Previous_Promotion_Sub_Win_Percent,Previous_Promotion_Decision_Win_Percent,Previous_Promotion_KO_Loss_Percent,Previous_Promotion_Sub_Loss_Percent,Previous_Promotion_Decision_Loss_Percent,Affiliation,Fighting_Out_Of,UFC_Fight_Count
0,AJ Dobson,7341.333333,0.0,0.0,59.000000,128.333333,83.666667,157.666667,0.476667,0.540000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error: [Errno 2] No such file or directory: 'd...,Error: [Errno 2] No such file or directory: 'd...,3
1,AJ Fletcher,7674.000000,0.0,0.0,38.250000,78.250000,55.000000,101.250000,0.477500,0.547500,...,1.000000,NaN,0.0,0.0,0.0,0.0,0.0,Gladiators Academy,"Lafayette, Louisiana",4
2,Aalon Cruz,548.000000,0.0,0.0,2.000000,12.000000,20.000000,29.000000,0.160000,0.680000,...,0.500000,0.0,0.0,NaN,0.0,0.0,NaN,Kaizen MMA & Yoga,"Tampa, Florida",1
3,Aaron Brink,8696.000000,0.0,0.0,0.000000,5.000000,2.000000,4.000000,0.000000,0.500000,...,0.500000,NaN,0.0,0.0,0.0,NaN,0.0,NaN,"San Diego, California, USA",1
4,Aaron Phillips,7681.666667,0.0,0.0,20.666667,36.333333,43.000000,71.000000,0.516667,0.603333,...,0.833333,NaN,NaN,NaN,NaN,0.0,0.0,Headkicks MMA,"Lafayette, Louisiana",3


In [25]:
# check distribution of UFC_Fight_Count
fighter_averages['UFC_Fight_Count'].describe()

count    2143.000000
mean        6.122259
std         6.107501
min         1.000000
25%         2.000000
50%         4.000000
75%         8.000000
max        47.000000
Name: UFC_Fight_Count, dtype: float64

### Add Body Size Features

In [26]:
# check for columns with words 'height' or 'Height', 'reach' or 'Reach', return list of them
height_cols = [col for col in fighter_averages.columns if 'height' in col.lower()]
reach_cols = [col for col in fighter_averages.columns if 'reach' in col.lower()]

size_cols = height_cols + reach_cols
size_cols

['A_Height',
 'B_Height',
 'Dif_Height',
 'A_Body_Length_to_Height_Ratio',
 'B_Body_Length_to_Height_Ratio',
 'Body_Length_to_Height_Ratio_Dif',
 'A_Reach',
 'B_Reach',
 'Dif_Reach',
 'A_Leg_Reach',
 'B_Leg_Reach',
 'Dif_Leg_Reach',
 'A_Reach_NA',
 'B_Reach_NA',
 'Reach_NA',
 'A_Leg_Reach_NA',
 'B_Leg_Reach_NA',
 'Leg_Reach_NA',
 'A_Body_Length_to_Reach_Ratio',
 'B_Body_Length_to_Reach_Ratio',
 'Body_Length_to_Reach_Ratio_Dif',
 'A_Body_Length_to_Leg_Reach_Ratio',
 'B_Body_Length_to_Leg_Reach_Ratio',
 'Body_Length_to_Leg_Reach_Ratio_Dif']

In [27]:
# new feature: Body Length (Height - Leg Reach)
fighter_averages['A_Body_Length'] = fighter_averages['A_Height'] - fighter_averages['A_Leg_Reach']
fighter_averages['B_Body_Length'] = fighter_averages['B_Height'] - fighter_averages['B_Leg_Reach']


# new feature: Body Length to Height Ratio
fighter_averages['A_Body_Length_to_Height_Ratio'] = fighter_averages['A_Body_Length'] / fighter_averages['A_Height']

# new feature: Body Length to Reach Ratio
fighter_averages['A_Body_Length_to_Reach_Ratio'] = fighter_averages['A_Body_Length'] / fighter_averages['A_Reach']

# new feature: Body Length to Leg Reach Ratio
fighter_averages['A_Body_Length_to_Leg_Reach_Ratio'] = fighter_averages['A_Body_Length'] / fighter_averages['A_Leg_Reach']



Get rid of affiliations (A_Affiliation, B_Affiliation) with errors, replace with "unknown"

In [28]:
# Get rid of affiliations (A_Affiliation, B_Affiliation) with errors, replace with "unknown"
# if affiliation contains "error", replace with "unknown"
fighter_averages['Affiliation'] = fighter_averages['Affiliation'].apply(lambda x: 'unknown' if 'error' in str(x).lower() else x)


In [29]:
today= datetime.date.today().strftime('%Y-%m-%d')
today

'2023-10-27'

In [30]:
fighter_averages.columns.to_list()

['Fighter_A',
 'Unnamed: 0.1',
 'A_Kd',
 'B_Kd',
 'A_Sig_strike_land',
 'A_Sig_strike_att',
 'B_Sig_strike_land',
 'B_Sig_strike_att',
 'A_Sig_strike_percent',
 'B_Sig_strike_percent',
 'A_Total_Strikes_land',
 'A_Total_Strikes_att',
 'B_Total_Strikes_land',
 'B_Total_Strikes_att',
 'A_Total_Strikes_percent',
 'B_Total_Strikes_percent',
 'A_Takedowns_land',
 'A_Takedowns_att',
 'B_Takedowns_land',
 'B_Takedowns_att',
 'A_Takedown_percent',
 'B_Takedown_percent',
 'A_Sub_Attempts_land',
 'A_Sub_Attempts_att',
 'B_Sub_Attempts_land',
 'B_Sub_Attempts_att',
 'A_Sub_Success_Percent',
 'B_Sub_Success_Percent',
 'A_Rev',
 'B_Rev',
 'A_Ctrl_time_min',
 'A_Ctrl_time_sec',
 'B_Ctrl_time_min',
 'B_Ctrl_time_sec',
 'A_Ctrl_time_tot',
 'B_Ctrl_time_tot',
 'A_Head_Strikes_land',
 'A_Head_Strikes_att',
 'B_Head_Strikes_land',
 'B_Head_Strikes_att',
 'A_Head_Strikes_percent',
 'B_Head_Strikes_percent',
 'A_Body_Strikes_land',
 'A_Body_Strikes_att',
 'B_Body_Strikes_land',
 'B_Body_Strikes_att',
 'A_B

In [31]:
# fighter_averages = pd.read_csv('data/final/aggregates/Current_Fighter_Averages_2023-10-23.csv')

In [32]:
to_drop= [
    'Unnamed: 0.1',
 'B_Height',
 'Dif_Height',
 'B_Reach',
 'Dif_Reach',
 'B_Leg_Reach',
 'Dif_Leg_Reach',
 'A_Reach_NA',
 'B_Reach_NA',
 'Reach_NA',
 'A_Leg_Reach_NA',
 'B_Leg_Reach_NA',
 'Leg_Reach_NA',
 'A_Fight_in_Typical_Weightclass',
 'B_Fight_in_Typical_Weightclass',
 'InFightData__Round',
 'B_Ape_Index',
 'B_Leg_Index',
 'B_Leg_to_Wing_Index',
 'win?',
 'favorite?',
 'A_Total_UFC_Fights',
 'B_Total_UFC_Fights',
 'Dif_Total_UFC_Fights',
 'Dif_UFC_Wins',
 'Dif_UFC_Losses',
 'Dif_UFC_Win_Percentage',
 'A_Last5_Win_Percentage',
 'B_Last5_Win_Percentage',
 'Dif_Last5_Win_Percentage',
 'A_Last3_Win_Percentage',
 'B_Last3_Win_Percentage',
 'Dif_Last3_Win_Percentage',
 'A_Win_By_KO_Percentage',
 'B_Win_By_KO_Percentage',
 'Dif_Win_By_KO_Percentage',
 'A_Loss_By_KO_Percentage',
 'B_Loss_By_KO_Percentage',
 'Dif_Loss_By_KO_Percentage',
 'A_Win_By_Decision_Percentage',
 'B_Win_By_Decision_Percentage',
 'Dif_Win_By_Decision_Percentage',
 'A_Loss_By_Decision_Percentage',
 'B_Loss_By_Decision_Percentage',
 'Dif_Loss_By_Decision_Percentage',
 'final_round_seconds',
 'Dif_UFC_Fight_Time_Seconds',
 'A_Martial_Art_Year_Started_1',
 'B_Martial_Art_Year_Started_1',
 'A_Martial_Art_Year_Started_2',
 'B_Martial_Art_Year_Started_2',
 'A_Martial_Art_Year_Started_3',
 'B_Martial_Art_Year_Started_3',
 'A_Previous_Promotion_Fights',
 'B_Previous_Promotion_Fights',
 'A_Previous_Promotion_Win_Percent',
 'B_Previous_Promotion_Win_Percent',
 'A_Previous_Promotion_KO_Win_Percent',
 'B_Previous_Promotion_KO_Win_Percent',
 'A_Previous_Promotion_Sub_Win_Percent',
 'B_Previous_Promotion_Sub_Win_Percent',
 'A_Previous_Promotion_Decision_Win_Percent',
 'B_Previous_Promotion_Decision_Win_Percent',
 'A_Previous_Promotion_KO_Loss_Percent',
 'B_Previous_Promotion_KO_Loss_Percent',
 'A_Previous_Promotion_Sub_Loss_Percent',
 'B_Previous_Promotion_Sub_Loss_Percent',
 'A_Previous_Promotion_Decision_Loss_Percent',
 'B_Previous_Promotion_Decision_Loss_Percent',
 'B_Body_Length',
 'Body_Length_Dif',
 'B_Body_Length_to_Height_Ratio',
 'Body_Length_to_Height_Ratio_Dif',
 'B_Body_Length_to_Reach_Ratio',
 'Body_Length_to_Reach_Ratio_Dif',
 'B_Body_Length_to_Leg_Reach_Ratio',
 'Body_Length_to_Leg_Reach_Ratio_Dif',
 'Ruleset',
 'A_Affiliation_Cluster',
 'B_Affiliation_Cluster',
 'A_Fighting_Out_Of_Cluster',
 'B_Fighting_Out_Of_Cluster',
 'A_Previous_Promotion_Cluster',
 'B_Previous_Promotion_Cluster',
 'Ape_Index_Dif',
 'Leg_Index_Dif',
 'Leg_to_Wing_Index_Dif',
 'A_Days_Since_Last_Fight',
 'B_Days_Since_Last_Fight',
 'Days_Since_Last_Fight_Dif',
]

In [33]:
# drop columns
fighter_averages.drop(columns=to_drop, inplace=True)

KeyError: "['A_Affiliation_Cluster', 'B_Affiliation_Cluster', 'A_Fighting_Out_Of_Cluster', 'B_Fighting_Out_Of_Cluster', 'A_Previous_Promotion_Cluster', 'B_Previous_Promotion_Cluster', 'Ape_Index_Dif', 'Leg_Index_Dif', 'Leg_to_Wing_Index_Dif', 'A_Days_Since_Last_Fight', 'B_Days_Since_Last_Fight', 'Days_Since_Last_Fight_Dif', 'Fighter_A_Odds_Implied_Prob', 'Fighter_B_Odds_Implied_Prob', 'Fighter_Odds_Implied_Prob_Dif'] not found in axis"

In [ ]:
# drop all columns that contain A_Opp or B_Opp
fighter_averages.drop(fighter_averages.filter(regex='A_Opp').columns, axis=1, inplace=True)
fighter_averages.drop(fighter_averages.filter(regex='B_Opp').columns, axis=1, inplace=True)

# drop all A_UFC or B_UFC columns
fighter_averages.drop(fighter_averages.filter(regex='A_UFC').columns, axis=1, inplace=True)
fighter_averages.drop(fighter_averages.filter(regex='B_UFC').columns, axis=1, inplace=True)

# drop all Dif_Opp columns
fighter_averages.drop(fighter_averages.filter(regex='Dif_Opp').columns, axis=1, inplace=True)



In [ ]:
# fix column names
# all columns starting with A_ should be renamed to Average_
# all columns starting with B_ should be renamed to Defense_Avg_

# get list of columns to rename
cols_to_rename = [col for col in fighter_averages.columns if col.startswith('A_') or col.startswith('B_')]
cols_to_rename

# rename columns
for col in cols_to_rename:
    if col.startswith('A_'):
        fighter_averages.rename(columns={col: col.replace('A_', 'Average_')}, inplace=True)
    elif col.startswith('B_'):
        fighter_averages.rename(columns={col: col.replace('B_', 'Defense_Avg_')}, inplace=True)




In [ ]:
# sums to get from fighter_sums

fighter_sums_to_get = [
 'A_Kd',
 'B_Kd',
 'A_Sig_strike_land',
 'A_Sig_strike_att',
 'B_Sig_strike_land',
 'B_Sig_strike_att',
 'A_Total_Strikes_land',
 'A_Total_Strikes_att',
 'B_Total_Strikes_land',
 'B_Total_Strikes_att',
 'A_Takedowns_land',
 'A_Takedowns_att',
 'B_Takedowns_land',
 'B_Takedowns_att',
 'A_Sub_Attempts_land',
 'A_Sub_Attempts_att',
 'B_Sub_Attempts_land',
 'B_Sub_Attempts_att',
 'A_Rev',
 'B_Rev',
 'A_Ctrl_time_tot',
 'B_Ctrl_time_tot',
 'A_Head_Strikes_land',
 'A_Head_Strikes_att',
 'B_Head_Strikes_land',
 'B_Head_Strikes_att',
 'A_Body_Strikes_land',
 'A_Body_Strikes_att',
 'B_Body_Strikes_land',
 'B_Body_Strikes_att',
 'A_Leg_Strikes_land',
 'A_Leg_Strikes_att',
 'B_Leg_Strikes_land',
 'B_Leg_Strikes_att',
 'A_Distance_Strikes_land',
 'A_Distance_Strikes_att',
 'B_Distance_Strikes_land',
 'B_Distance_Strikes_att',
 'A_Clinch_Strikes_land',
 'A_Clinch_Strikes_att',
 'B_Clinch_Strikes_land',
 'B_Clinch_Strikes_att',
 'A_Ground_Strikes_land',
 'A_Ground_Strikes_att',
 'B_Ground_Strikes_land',
 'B_Ground_Strikes_att'
]

In [ ]:
# bring up fighter sums
fighter_sums

# keep only the index column and the figher_sums_to_get columns
fighter_sums = fighter_sums[fighter_sums_to_get]

fighter_sums.head()


In [ ]:
# add fighter_counts to fighter_sums
fighter_sums = pd.merge(fighter_sums, fighter_counts, on='Fighter_A', how='left')
fighter_sums.head()

In [ ]:
# rename columns
fighter_sums.rename(columns={'A_Kd': 'Total Knockdowns', 'B_Kd': 'Total Times Knocked Down', 
                             'A_Sig_strike_land' : 'Signiture Strikes Landed', 'A_Sig_strike_att' : 'Signiture Strikes Attempted',
                                'B_Sig_strike_land' : 'Signiture Strikes Landed Against', 'B_Sig_strike_att' : 'Signiture Strikes Attempted Against',
                                'A_Total_Strikes_land' : 'Total Strikes Landed', 'A_Total_Strikes_att' : 'Total Strikes Attempted',
                                'B_Total_Strikes_land' : 'Total Strikes Landed Against', 'B_Total_Strikes_att' : 'Total Strikes Attempted Against',
                                'A_Takedowns_land' : 'Takedowns Landed', 'A_Takedowns_att' : 'Takedowns Attempted',
                                'B_Takedowns_land' : 'Takedowns Landed Against', 'B_Takedowns_att' : 'Takedowns Attempted Against',
                                'A_Sub_Attempts_land' : 'Submission Attempts', 'A_Sub_Attempts_att' : 'Submission Attempts',
                                'B_Sub_Attempts_land' : 'Submission Attempts Against', 'B_Sub_Attempts_att' : 'Submission Attempts Against',
                                'A_Rev' : 'Reversals', 'B_Rev' : 'Reversals Against',
                                'A_Ctrl_time_tot' : 'Control Time', 'B_Ctrl_time_tot' : 'Control Time Against',
                                'A_Head_Strikes_land' : 'Head Strikes Landed', 'A_Head_Strikes_att' : 'Head Strikes Attempted',
                                'B_Head_Strikes_land' : 'Head Strikes Landed Against', 'B_Head_Strikes_att' : 'Head Strikes Attempted Against',
                                'A_Body_Strikes_land' : 'Body Strikes Landed', 'A_Body_Strikes_att' : 'Body Strikes Attempted',
                                'B_Body_Strikes_land' : 'Body Strikes Landed Against', 'B_Body_Strikes_att' : 'Body Strikes Attempted Against',
                                'A_Leg_Strikes_land' : 'Leg Strikes Landed', 'A_Leg_Strikes_att' : 'Leg Strikes Attempted',
                                'B_Leg_Strikes_land' : 'Leg Strikes Landed Against', 'B_Leg_Strikes_att' : 'Leg Strikes Attempted Against',
                                'A_Distance_Strikes_land' : 'Distance Strikes Landed', 'A_Distance_Strikes_att' : 'Distance Strikes Attempted',
                                'B_Distance_Strikes_land' : 'Distance Strikes Landed Against', 'B_Distance_Strikes_att' : 'Distance Strikes Attempted Against',
                                'A_Clinch_Strikes_land' : 'Clinch Strikes Landed', 'A_Clinch_Strikes_att' : 'Clinch Strikes Attempted',
                                'B_Clinch_Strikes_land' : 'Clinch Strikes Landed Against', 'B_Clinch_Strikes_att' : 'Clinch Strikes Attempted Against',
                                'A_Ground_Strikes_land' : 'Ground Strikes Landed', 'A_Ground_Strikes_att' : 'Ground Strikes Attempted',
                                'B_Ground_Strikes_land' : 'Ground Strikes Landed Against', 'B_Ground_Strikes_att' : 'Ground Strikes Attempted Against',
                                'UFC_Fight_Count' : 'UFC Fight Count'}, inplace=True)

fighter_sums.head()

In [ ]:
# check dtypes
fighter_sums.dtypes

In [ ]:
# calculate Accuracy Percentages
# Signiture Strike Accuracy
fighter_sums['Signiture Strike Accuracy'] = fighter_sums['Signiture Strikes Landed'] / fighter_sums['Signiture Strikes Attempted']
fighter_sums['Signiture Strike Accuracy Against'] = fighter_sums['Signiture Strikes Landed Against'] / fighter_sums['Signiture Strikes Attempted Against']

# Total Strike Accuracy
fighter_sums['Total Strike Accuracy'] = fighter_sums['Total Strikes Landed'] / fighter_sums['Total Strikes Attempted']
fighter_sums['Total Strike Accuracy Against'] = fighter_sums['Total Strikes Landed Against'] / fighter_sums['Total Strikes Attempted Against']

# Takedown Accuracy
fighter_sums['Takedown Accuracy'] = fighter_sums['Takedowns Landed'] / fighter_sums['Takedowns Attempted']
fighter_sums['Takedown Accuracy Against'] = fighter_sums['Takedowns Landed Against'] / fighter_sums['Takedowns Attempted Against']

# Head Strike Accuracy
fighter_sums['Head Strike Accuracy'] = fighter_sums['Head Strikes Landed'] / fighter_sums['Head Strikes Attempted']
fighter_sums['Head Strike Accuracy Against'] = fighter_sums['Head Strikes Landed Against'] / fighter_sums['Head Strikes Attempted Against']

# Body Strike Accuracy
fighter_sums['Body Strike Accuracy'] = fighter_sums['Body Strikes Landed'] / fighter_sums['Body Strikes Attempted']
fighter_sums['Body Strike Accuracy Against'] = fighter_sums['Body Strikes Landed Against'] / fighter_sums['Body Strikes Attempted Against']

# Leg Strike Accuracy
fighter_sums['Leg Strike Accuracy'] = fighter_sums['Leg Strikes Landed'] / fighter_sums['Leg Strikes Attempted']
fighter_sums['Leg Strike Accuracy Against'] = fighter_sums['Leg Strikes Landed Against'] / fighter_sums['Leg Strikes Attempted Against']

# Distance Strike Accuracy
fighter_sums['Distance Strike Accuracy'] = fighter_sums['Distance Strikes Landed'] / fighter_sums['Distance Strikes Attempted']
fighter_sums['Distance Strike Accuracy Against'] = fighter_sums['Distance Strikes Landed Against'] / fighter_sums['Distance Strikes Attempted Against']

# Clinch Strike Accuracy
fighter_sums['Clinch Strike Accuracy'] = fighter_sums['Clinch Strikes Landed'] / fighter_sums['Clinch Strikes Attempted']
fighter_sums['Clinch Strike Accuracy Against'] = fighter_sums['Clinch Strikes Landed Against'] / fighter_sums['Clinch Strikes Attempted Against']

# Ground Strike Accuracy
fighter_sums['Ground Strike Accuracy'] = fighter_sums['Ground Strikes Landed'] / fighter_sums['Ground Strikes Attempted']
fighter_sums['Ground Strike Accuracy Against'] = fighter_sums['Ground Strikes Landed Against'] / fighter_sums['Ground Strikes Attempted Against']


In [ ]:
fighter_sums.head()

In [ ]:
# add fighter_sums to fighter_averages
fighter_averages = pd.merge(fighter_averages, fighter_sums, on='Fighter_A', how='left')
fighter_averages.head()

In [ ]:
# save as current fighter averages
fighter_averages.to_csv('data/final/aggregates/Current_Fighter_Averages_'+ today +'.csv', index=False)